# Iteration

**Learning Objectives:** Understand and apply different approaches to iteration in Python, including iterators, generators, list/dict comprehensions, and functional approaches.

Interation is a powerful abstraction in Python that is an important part of working with data efficiently. The basic idea is to express computations on the elements in some sort of container. The simplest example of iteration in Python is a `for` loop:

In [1]:
for i in range(4):
    print(i)

0
1
2
3


In [2]:
range(4)

range(0, 4)

In this example, `range(4)` returns an *iterator* and the `for` loop performs *iteration* on the elements of that *iterator*. A Python `list` can also be used in a `for` loop:

In [3]:
for state in ['CA', 'OR', 'NY', 'MA']:
    print(state)

CA
OR
NY
MA


When a Python `dict` is iterated over, the keys will be returned:

In [4]:
for field in {'name': 'Bart Simpson', 'age': 10}:
    print(field)

age
name


Many container objects can be iterated over in this manner including the `range` object, `list`, `tuple`, `dict`, `set` and the lines of a file.

## Iterators

The idea of an iterator is formalized in Python through the iterator protocol, which is described here:

https://docs.python.org/3.4/library/stdtypes.html#iterator-types

The basic idea is this:

* A container object that follows the iterator protocol has a special `__iter__` method that returns an iterator object for the container.
* The iterator object itself has:
  - An `__iter__` method that returns itself.
  - A `__next__` method that will either return the next element in the container, or raise `StopIteration` if there are no remaining elements.

Python offers two related public functions for working with iterators, `iter` and `next`. These can be illustrated using a simple list:

In [5]:
l = [0,1,2,3]

In [6]:
hasattr(l, '__iter__')

True

The `iter` function will return an iterator for the list:

In [14]:
li = iter(l)
li

In [15]:
hasattr(li, '__next__') and hasattr(li, '__iter__')

True

Calling the `next` function will keep returning subsequent elements from the iterator:

In [16]:
next(li)

0

In [17]:
li2 = iter(li)

In [18]:
next(li2)

1

In [9]:
next(li)

0

In [10]:
next(li)

1

In [11]:
next(li)

2

In [12]:
next(li)

3

When there are no remaining elements to iterate over, `next` will raise `StopIteration`:

In [13]:
next(li)

StopIteration: 

The iterator protocol is used underneath the hood to implement `for` loops in Python. Thus, the following `for` loop:

In [19]:
for i in range(5):
    print(i)

0
1
2
3
4


is roughly equivalent to the following `while` loop that explicitly uses the iterator protocol:

In [20]:
seq = range(5)
it = iter(seq)

while True:
    try:
        i = next(it)
    except StopIteration:
        break
    else:
        print(i)

0
1
2
3
4


One of the most important points about iterators is that they are memory efficient. This is because iterators are not required to have all elements of the iterator in memory at the same time. An example of this is the builtin `range` function. The `range` function returns an iterator rather than a concrete list and is thus extremely fast and memory efficient. This uses $\mathcal{O}(1)$ memory:

In [21]:
%timeit range(10000)

The slowest run took 6.16 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 193 ns per loop


Converting that `range` object to a concrete list uses $\mathcal{O}(10,000)$ memory and is significantly slower:

In [22]:
%timeit list(range(10000))

10000 loops, best of 3: 141 µs per loop


<div class="alert alert-info">Recommendation: Favor abstract iterators over concrete sequences (`list`, `tuple`, `dict`).</div>

## Generators

Generators provide an elegant and simple way of creating new iterators using Python functions. Generators are described in detail here:

https://docs.python.org/3.4/library/stdtypes.html#generator-types

A generator:

* Is a regular Python function.
* Uses `yield` rather than `return` to return values.
* Can `yield` multiple values.
* Returns an iterator when called.

Here is a simple function that yields two values:

In [23]:
def foobar():
    yield 'foo'
    yield 'bar'

Calling the generator returns an iterator:

In [24]:
fb = foobar()
fb

<generator object foobar at 0x7f97707c6d38>

In [25]:
hasattr(fb, '__next__') and hasattr(fb, '__iter__')

True

These iterators can be used anywhere an iterator is expected:

In [26]:
for thing in foobar():
    print(thing)

foo
bar


In [27]:
list(foobar())

['foo', 'bar']

A generator can yield infinitely many values:

In [28]:
import time

def infinite_clock():
    while True:
        time.sleep(1.0)
        yield 'tick'

In [29]:
for i in infinite_clock():
    print(i)

tick
tick
tick
tick
tick
tick
tick
tick


KeyboardInterrupt: 

Here is a generator that generates a repeated constant in $\mathcal{O}(1)$ memory:

In [30]:
def constant(n, m):
    """Yield n, m times."""
    count = 0
    while count < m:
        yield n
        count += 1

In [31]:
for c in constant(5, 10):
    print(c)

5
5
5
5
5
5
5
5
5
5


In [32]:
10*[5]

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

This is much more efficient that using a list such as `m*[n]`:

## List Comprehensions

In many cases, we do want to work with concrete lists. The *list comprehension* provides an efficient way of creating lists.

To see where list comprehensions are useful, let's look at a common pattern used with lists. You will often find yourself creating an empty list and then appending elements to it in a `for` loop:

In [33]:
import random

result = []
for i in range(10):
    result.append(random.random())
result

[0.6180279296882156,
 0.43842852639373175,
 0.4675292400966653,
 0.5602200631088471,
 0.197638307005509,
 0.7347577387117097,
 0.9446941239263652,
 0.10035258831475258,
 0.12839583571787494,
 0.2752449972863712]

List comprehensions make this pattern extremely simple:

In [35]:
[i*random.random() for i in range(10)]

[0.0,
 0.3562359510372546,
 1.5866165224098074,
 1.47889192441732,
 3.4011502553965838,
 1.9331747045047027,
 0.5047483372110961,
 1.509902459549605,
 7.97490216680554,
 6.073436549439883]

In addition to being simple from the code perspective, list comprehensions are usually faster than for loops as this example demonstrates:

In [36]:
def vector_add_slow(x, y):
    n = len(x)
    result = []
    for i in range(n):
        result.append(x[i]+y[i])
    return result

In [37]:
def vector_add_fast(x, y):
    return [x_i+y_i for x_i, y_i in zip(x,y)] # we will learn about zip shortly

In [38]:
x = [random.random() for i in range(1000)]
y = [random.random() for i in range(1000)]

In [39]:
%timeit vector_add_slow(x,y)

10000 loops, best of 3: 108 µs per loop


In [40]:
%timeit vector_add_fast(x,y)

10000 loops, best of 3: 60.6 µs per loop


List comprehensions also allow nested loops and tests:

In [42]:
[(i,j,i*j) for i in range(4) for j in range(4) if i!=j]

[(0, 1, 0),
 (0, 2, 0),
 (0, 3, 0),
 (1, 0, 0),
 (1, 2, 2),
 (1, 3, 3),
 (2, 0, 0),
 (2, 1, 2),
 (2, 3, 6),
 (3, 0, 0),
 (3, 1, 3),
 (3, 2, 6)]

<div class="alert alert-info">Recommendation: Prefer list comprehensions to `for` loops. </div>

## Generator expressions

List comprehensions are elegant and fast. However, list comprehensions are inefficient from a memory perspective as they create a concrete list, where each element of the list exists in memory at the same time. A *generator expression* offers a syntax similar to that of a list comprehension but with the memory efficiency of a generator. Generator expressions will give you Python superpowers.

Here is a simple example that performs the sum of 10,000 random numbers. This version is $\mathcal{O}(10,000)$ in memory:

In [43]:
%%timeit
x = []
for i in range(10000):
    x.append(random.random())
result = 0.0
for element in x:
    result += element

1000 loops, best of 3: 1.41 ms per loop


By using a list comprehension, we can speed up the execution time, but it is still $\mathcal{O}(10,000)$ in memory:

In [44]:
%%timeit
x = [random.random() for i in range(10000)]
result = sum(x)

1000 loops, best of 3: 899 µs per loop


A generator expression looks exactly like a list comprehension, but with the `[]` replaced by `()`. Here is the generator expression version that is $\mathcal{O}(1)$ and nearly as fast as the list comprehension version:

In [45]:
%%timeit
x = (random.random() for i in range(10000))
result = sum(x)

1000 loops, best of 3: 925 µs per loop


If a function takes *single* argument that is an iterator, you can pass a generator expression with out the extra parentheses:

In [ ]:
%%timeit
result = sum(random.random() for i in range(10000))

This version is easier to read and faster that the initial `for` loop/list version and is $\mathcal{O}(1)$ in memory.

## Dict comprehensions

A dict comprehension is like a list comprehension, but for creating concrete `dict` objects. It uses the syntax `{k: v for ...}`:

In [46]:
letters = {i : chr(65+i) for i in range(26)}
letters

{0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
 10: 'K',
 11: 'L',
 12: 'M',
 13: 'N',
 14: 'O',
 15: 'P',
 16: 'Q',
 17: 'R',
 18: 'S',
 19: 'T',
 20: 'U',
 21: 'V',
 22: 'W',
 23: 'X',
 24: 'Y',
 25: 'Z'}

A dict comprehension is a simple way of inverting the keys and values of a dict:

In [47]:
{v: k for k, v in letters.items()}

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

## Functional approaches to iteration

Python also provides a number of functions that are helpful in performing iteration.

The `zip` function enables you to "zip" two iterators together:

In [48]:
a = range(10)
b = range(10,0,-1)

In [51]:
list(a)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [52]:
list(b)

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [53]:
for o in zip(a,b):
    print(o)

(0, 10)
(1, 9)
(2, 8)
(3, 7)
(4, 6)
(5, 5)
(6, 4)
(7, 3)
(8, 2)
(9, 1)


If the iterators passed to `zip` have different lengths, the result will have the shortest length:

In [54]:
c = range(5)

In [55]:
for o in zip(a, c):
    print(o)

(0, 0)
(1, 1)
(2, 2)
(3, 3)
(4, 4)


The `enumerate` function consumes an iterator of values and returns a new one that has pairs of `(index, value)`. This can be very useful in helping you to avoid things like `range(len(x))`:

In [56]:
states = ['CA', 'OR', 'WA', 'NV', 'NY']

In [58]:
for state in state:
    print(state)

M
A


In [59]:
for i, state in enumerate(states):
    print(i, state)

0 CA
1 OR
2 WA
3 NV
4 NY


The `map` function provides an efficient way of applying a function to each element of an iterator:

In [62]:
 r = map(lambda x: x**2, range(10))

In [63]:
list(r)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

The object returned by `map` is an abstract iterator, and is thus memory efficient. However, in most cases, a generator expression is simpler and just as efficient:

In [65]:
(x**2 for x in range(10))

<generator object <genexpr> at 0x7f977079cd80>

In [66]:
list(_)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

The `reduce` function from the `functools` package is often used along with `map` and provides a clean way to "reduce" a sequence to a scalar value by applying a binary function sequentially to elements of the list.

In [67]:
from functools import reduce

In [68]:
reduce?

Docstring:
reduce(function, sequence[, initial]) -> value

Apply a function of two arguments cumulatively to the items of a sequence,
from left to right, so as to reduce the sequence to a single value.
For example, reduce(lambda x, y: x+y, [1, 2, 3, 4, 5]) calculates
((((1+2)+3)+4)+5).  If initial is present, it is placed before the items
of the sequence in the calculation, and serves as a default when the
sequence is empty.
Type:      builtin_function_or_method

For example, this function computes the factorial of an integer using `reduce`:

In [70]:
def factorial(n):
    return reduce(lambda x,y: x*y, range(n,1,-1))

In [71]:
assert factorial(10)==10*9*8*7*6*5*4*3*2*1

Note that, while the `map` and `reduce` functions described here are *related* to the [MapReduce](http://static.googleusercontent.com/media/research.google.com/es/us/archive/mapreduce-osdi04.pdf) algorithm invented at Google, there are significant differences.